# ADA Project
## Modeling

## 4. LSTM

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

# Load and preprocess the data
data = pd.read_csv('../Data/merged_data.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)
data = data[data['animal-type'] == 'Wolf']

# Define grid search parameters
learning_rates = [1e-5, 1e-4, 1e-3]
nodes = [32, 64, 128, 256] # high number of nodes because all best models tested have between 64 to 256 nodes
layers = [3, 5, 7] # only 3, 5, 7 to avoid too complex models and overfitting
batch_size = 32
epochs = 10 # due to computational time, I can only train for 10 epochs and 32 batch size (running on a CPU and this code takes already hours to run on my machine)
patience = 3
time_steps = 20 # different lags have been tested and 20 seems to be the best tradeoff between accuracy and avoiding overfitting

# Prepare a dictionary to store the results for each wolf
results = {}

# Function to create overlapping windows
def create_overlapping_windows(data, time_steps):
    X, Y = [], [] 
    for i in range(len(data) - time_steps):
        X.append(data.iloc[i:i+time_steps].values)
        Y.append(data.iloc[i+time_steps].values)
    return np.array(X), np.array(Y)

# Process each wolf's data
for wolf_id, group in data.groupby('individual-id'):
    print(f"Processing wolf {wolf_id}")
    
    # Create overlapping windows
    X, y = create_overlapping_windows(group[['location-lat', 'location-long']], time_steps)
    
    # Scale the features
    scaler_X = MinMaxScaler(feature_range=(0, 1))
    scaler_y = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler_X.fit_transform(X.reshape(-1, 2)).reshape(X.shape)
    y_scaled = scaler_y.fit_transform(y)
    
    # Split the data into training and testing
    n_train = int(0.8 * len(X_scaled))
    X_train_scaled, y_train_scaled = X_scaled[:n_train], y_scaled[:n_train]
    X_test_scaled, y_test_scaled = X_scaled[n_train:], y_scaled[n_train:]

    # Initialize the best score to a high value
    best_score = np.inf
    best_params = {}

    # Grid search for hyperparameters
    for lr in learning_rates:
        for node in nodes:
            for layer in layers:
                # Define the model
                model = Sequential()
                
                model.add(LSTM(node, return_sequences=True, input_shape=(time_steps, 2), activation='relu')) # First RNN layer
                
                for _ in range(layer - 2): # -2 because first and last layers are already added
                    model.add(LSTM(node, return_sequences=True, activation='relu'))
                model.add(LSTM(node, activation='relu'))  # Last RNN layer
                
                model.add(Dense(2, activation='linear'))  # Output layer
                
                # Compile the model
                optimizer = Adam(learning_rate=lr)
                model.compile(loss='mse', optimizer=optimizer)
                
                # Fit the model with early stopping
                early_stopping = EarlyStopping(monitor='val_loss', patience=patience)
                history = model.fit(
                    X_train_scaled, y_train_scaled, 
                    validation_split=0.2,  # Using 20% of the training data for validation
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[early_stopping], 
                    verbose=0
                )

                # Evaluate the model on the test data
                score = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
                print(f"Model with LR: {lr}, Nodes: {node}, Layers: {layer} has MSE: {score}")

                # Update best model if the score improved
                if score < best_score:
                    best_score = score
                    best_params = {'lr': lr, 'nodes': node, 'layers': layer}
                    best_model = model

    # Forecast future values using the last available input from training
    current_input = X_train_scaled[-1].reshape(1, time_steps, 2)
    predictions = []
    for _ in range(len(y_test_scaled)):
        next_prediction = best_model.predict(current_input)
        predictions.append(next_prediction[0])
        current_input = np.roll(current_input, -1, axis=1)
        current_input[0, -1, :] = next_prediction[0]

    # Inverse transform predictions to original scale
    predictions_scaled_back = scaler_y.inverse_transform(predictions)
    mse = mean_squared_error(scaler_y.inverse_transform(y_test_scaled), predictions_scaled_back)
    
    results[wolf_id] = {
        'mse': mse,
        'predictions': predictions_scaled_back,
        'actual': scaler_y.inverse_transform(y_test_scaled),
        'best_params': best_params
    }
    
    # Save the best model for each wolf
    best_model.save(f'../Results/RNN results/best_model_for_wolf_{wolf_id}.keras')

# Output the results for each wolf
for wolf_id, res in results.items():
    print(f"Wolf ID: {wolf_id}, Test set MSE: {res['mse']}, Best Params: {res['best_params']}")

2024-04-30 11:25:29.168803: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xe

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xe

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xe

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception